In [ ]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [ ]:
from google.cloud.bigquery.job import QueryJobConfig

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    row_number, 
    col, 
    lit, 
    count, 
    log, 
    exp, 
    sum as spark_sum
)
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, date, timedelta
from typing import List, Dict

In [ ]:
print(f'current_dt: {current_dt}')
print(f'state: {state}')
print(f'long_duration: {long_duration}')


In [ ]:
execution_dt = datetime.strptime(current_dt, '%Y-%m-%d')
execution_dt_one_ago = (execution_dt - timedelta(days=1)).strftime('%Y-%m-%d')
execution_dt_next = (execution_dt + timedelta(days=1))
current_dt_next = execution_dt_next.strftime('%Y-%m-%d')

In [ ]:
db_name = 'adot_reco_dev'
project_id = 'skt-datahub'

In [ ]:
bq_client = get_bigquery_client()

In [ ]:
import requests
import pandas as pd
import json

In [ ]:
url = "http://insight.nugu.co.kr/psionic/v1/api/export/edf_1718241386286"
headers = {
    "Content-Type": "application/json", 
    "token": nudge_token
}

In [ ]:
response = requests.post(url, headers=headers, data="{}")
if int(response.status_code) != 200:
    raise RuntimeError(f"api request failed: {response.status_code}")


In [ ]:
api_text = json.loads(response.text)

In [ ]:
def get_api_cols(infos:List[Dict]):
    result = list()
    for info in infos:
        result.append(info['name'])
    return result

In [ ]:
api_info = api_text['info']
api_data = api_text['data']
col_names = get_api_cols(api_info)

In [ ]:
api_final_result = []
for api_datum in api_data:
    temp_dict = dict()
    data = api_datum['row']
    for col_name, datum in zip(col_names, data):
        if col_name == 'event_id_description' and (datum is None or datum ==""):
            temp_dict = None
            break
        elif col_name =='variable':
            datum = datum['titleTextPolite']
        else:
            temp_dict[col_name] = datum
    if temp_dict is not None:
        temp_dict['dt'] = current_dt_next
        api_final_result.append(temp_dict)

In [ ]:
api_data = pd.DataFrame(api_final_result)

In [ ]:
table_nm = 'nudge_offering_api_table'

In [ ]:
table_exists = bq_table_exists(table=f'{db_name}.{table_nm}', project_id=project_id)

In [ ]:
if not table_exists:
    bq_client.query(f"""
        CREATE TABLE IF NOT EXISTS {db_name}.{table_nm}(
            product STRING,
            scenario_id STRING,
            nudge_id STRING,
            event_id,
            event_id_description,
            variable,
            nudge_title,
            nudge_description,
            mandatory_yn
            dt Date
        )
    """).result()
    print(f"생성된 테이블 : {db_name}.{table_nm}")

else:
    table_ref = bq_client.dataset(db_name).table(table_nm)
    table = bq_client.get_table(table_ref)

    # Get the schema
    schema = table.schema

    #Extract column names
    column_names = [field.name for field in schema]

    new_columns = []
    for col_nm in col_names:
        if col_nm not in column_names:
            new_columns.append(col_nm)
    
    if new_columns:
        raise TypeError(f"new column:{new_columns} added from api")

In [ ]:
pandas_to_bq(pd_df=api_data, destination=f'{project_id}.{db_name}.{table_nm}')